In [ ]:
%pip install tensorflow keras 

In [ ]:
import tensorflow as tf

#picks first gpu for training
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True) 
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

#preparing images for resnet50
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range= 20,
    width_shift_range=0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip= True,
    fill_mode = 'nearest'
)

2025-11-25 19:16:15.313362: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
#only for test,valid dataset
test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [6]:
import os 
output_folder = os.path.join('dataset_split')

train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')

image_size = (256,256)
batch_size = 32

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle=False
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False #TODO ZMENIT PRI DALSIM TESTOVANI NA TRUE
)

Found 4560 images belonging to 4 classes.
Found 980 images belonging to 4 classes.
Found 1014 images belonging to 4 classes.


In [ ]:
from keras.applications.resnet import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3)) # 3 = rgb

base_model.trainable= False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras import layers, models

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(129, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(30, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy']) # TODO zmenit categorical_crossentropy -> nemusi se v metrics convertovat loss na categorical a je to efektivni na pamet

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=12,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        'best_model.keras',
        save_best_only=True,
        monitor='val_loss'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5
    ),
    CSVLogger('training_history.csv')
]

In [ ]:
model.fit(train_data, epochs =75, validation_data=val_data, callbacks=callbacks, use_multiprocessing=True, workers=6, max_queue_size=8)
model.save("base_model.keras")


In [ ]:
#save for metrics
import numpy as np

X_test = np.concatenate([test_data[i][0] for i in range(len(test_data))])
y_test = np.array(test_data.labels)
labels = test_data.class_indices
print(labels) #TODO klidne smazat

np.save("saved_data/X_test.npy", X_test)
np.save("saved_data/y_test.npy", y_test)

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
